# Check: initial downloads

Make maps to see that rail/ferry/brt all show up correctly.

In [ ]:
import geopandas as gpd
import pandas as pd

from IPython.display import Markdown

from segment_speed_utils import helpers
from update_vars import analysis_date, TEMP_GCS

# Map arguments
TILES = "Carto DB Positron"

In [ ]:
def make_map(gdf, plot_col):
    date_cols = [c for c in gdf.columns if 
                 gdf[c].dtype == 'datetime64[ns]']

    gdf = gdf.drop(columns = date_cols)
        
    m = gdf.explore(plot_col, categorical = True, tiles = TILES)
    
    display(m)

## After `A1_download_rail_ferry_brt`

* There are some stops to remove. 
* Once finalized, can run `A2_combine_stops`

In [ ]:
import A1_rail_ferry_brt_stops as rail_ferry_brt

### LA Metro (182)

In [ ]:
brt_stops = gpd.read_parquet(f"{TEMP_GCS}brt_stops.parquet")

In [ ]:
brt_stops_filtered = rail_ferry_brt.additional_brt_filtering_out_stops(
    brt_stops
)

In [ ]:
name = "LA Metro Bus Schedule"
make_map(brt_stops[brt_stops.name==name], "route_id")

In [ ]:
make_map(brt_stops_filtered[brt_stops_filtered.name==name], 
         "route_id")

### SF Muni (282)

In [ ]:
name = "Bay Area 511 Muni Schedule"
make_map(brt_stops[brt_stops.name==name], "route_id")

In [ ]:
make_map(brt_stops_filtered[brt_stops_filtered.name==name], 
         "route_id")

### AC Transit (4)

In [ ]:
name = "Bay Area 511 AC Transit Schedule"
make_map(brt_stops[brt_stops.name==name], "route_id")

## After `A3_rail_ferry_brt_extract`

#### BRT Service likely meeting [PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit) definition:

* LA Metro Orange, Silver excluding street running (stop flags only)
* ~~Omnitrans sbX, all stops (curbside stations are well-defined, with fare prepayment)~~
    * insufficient frequency 5/16
* AC Transit Tempo, all stops (curbside stations are well-defined, with fare prepayment)

In [ ]:
stops = rail_ferry_brt.get_rail_ferry_brt_extract()

In [ ]:
operators = helpers.import_scheduled_trips(
    analysis_date,
    columns = ["feed_key", "name"],
    get_pandas = True
).rename(columns = {"feed_key": "feed_key_primary"})

In [ ]:
hqta_types = list(stops.hqta_type.unique())

In [ ]:
for i in hqta_types:
    subset = stops[stops.hqta_type==i].merge(
        operators,
        on = "feed_key_primary"
    )
    
    display(Markdown(f"### HQTA Type: {i}"))
    
    make_map(subset, "name")